In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0527 15:40:54.494622 140715147655040 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [3]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 3.3MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [5]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'context_free_model'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: context_free_model *****


#Data

In [6]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks


In [0]:
import pandas as pd
train_data = pd.read_csv("train_sarc_proc_bal.csv")
test_data = pd.read_csv("test_sarc_proc_bal.csv")

train.columns

In [0]:
train_responses1 = train_data[['Response 1','Ancestor','Label 1']].rename({'Response 1':'Response','Label 1':'Label'}, axis='columns')
train_responses2 = train_data[['Response 2','Ancestor','Label 2']].rename({'Response 2':'Response','Label 2':'Label'}, axis='columns')
train = pd.concat([train_responses1,train_responses2],sort=False)

test_responses1 = test_data[['Response 1','Ancestor','Label 1']].rename({'Response 1':'Response','Label 1':'Label'}, axis='columns')
test_responses2 = test_data[['Response 2','Ancestor','Label 2']].rename({'Response 2':'Response','Label 2':'Label'}, axis='columns')
test = pd.concat([test_responses1, test_responses2],sort=False)

In [0]:
DATA_COLUMN = 'Response'
LABEL_COLUMN = 'Label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [-1, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [11]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0527 15:41:51.493089 140715147655040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0527 15:41:53.861562 140715147655040 saver.py:1483] Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [12]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [13]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 13668


I0527 15:41:56.222014 140715147655040 run_classifier.py:774] Writing example 0 of 13668


INFO:tensorflow:*** Example ***


I0527 15:41:56.225440 140715147655040 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0527 15:42:00.762332 140715147655040 run_classifier.py:466] input_mask: 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0527 15:42:00.764008 140715147655040 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


I0527 15:42:00.765606 140715147655040 run_classifier.py:468] label: 1 (id = 1)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

import numpy as np
def getPredictions(df, data="Response", label="Label"):
  print(label_list)
  labels = [-1, 1]
  input_examples = df.apply(lambda x:bert.run_classifier.InputExample(guid=None, 
                                                                 text_a = x[data], 
                                                                 text_b = None, label = x[label]),axis=1)
  
  input_features = bert.run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = bert.run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = list(estimator.predict(predict_input_fn))
  probs = np.array([pred['probabilities'] for pred in predictions])
  preds = [int(labels[pred['labels']]) for pred in predictions]
  df['probs1'] = pd.Series(probs[:,0])
  df['probs2'] = pd.Series(probs[:,1])
  df['labels'] = pd.Series(preds)
  
  return df
#   return [(prediction['labels']) for prediction in zip(predictions)]


for i in range(5):
  print("Iteration nr "+str(i))
  model_fn = model_fn_builder(
    num_labels=len(label_list),
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps)

  estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    params={"batch_size": BATCH_SIZE})

  print(f'Beginning Training!')
  current_time = datetime.now()
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print("Training took time ", datetime.now() - current_time)
  import numpy as np
  getPredictions(test_responses1)
  getPredictions(test_responses2)
  
  # train_pred_labels = 2*(classifier.predict(train_responses1)>classifier.predict(train_responses2))-1
  # train_expect_labels = train_data['Label 1']

  # predict which comment is more sarcastic out of two
  test_pred_labels = 2*(np.exp(test_responses1['probs2'])>np.exp(test_responses2['probs2']))-1
  # expected label is the Label 1 -> if first response is classified as sarcastic, the label would be 1, otherwise -1
  test_expect_labels = test_data['Label 1']



  # print('\tTrain acc: ', (train_pred_labels == train_expect_labels).sum() / train_pred_labels.shape[0])
  print('\tTest acc: ', (test_pred_labels == test_expect_labels).sum() / test_pred_labels.shape[0])
  test_data_with_preds = test_data[['Response 1','Response 2','Ancestor','Label 1']]
  test_data_with_preds['preds'] = test_pred_labels
  file_name = "results_context_free "+str(i)+".csv"
  test_data_with_preds.to_csv(file_name,index=False)

Iteration nr 0
INFO:tensorflow:Using default config.


I0527 15:45:52.177154 140715147655040 estimator.py:1739] Using default config.


W0527 15:45:52.180536 140715147655040 estimator.py:1760] Using temporary folder as model directory: /tmp/tmpjnfohov9


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjnfohov9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffa58887470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0527 15:45:52.183520 140715147655040 estimator.py:201] Using config: {'_model_dir': '/tmp/tmpjnfohov9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffa58887470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

I0527 16:38:09.743555 140715147655040 basic_session_run_hooks.py:594] Saving checkpoints for 681 into /tmp/tmptw_f2tre/model.ckpt.


INFO:tensorflow:global_step/sec: 1.05506


I0527 16:38:36.522675 140715147655040 basic_session_run_hooks.py:680] global_step/sec: 1.05506


INFO:tensorflow:loss = 0.1482125, step = 700 (94.782 sec)


I0527 16:38:36.526632 140715147655040 basic_session_run_hooks.py:247] loss = 0.1482125, step = 700 (94.782 sec)


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0526 20:53:17.631509 140692488898432 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0526 20:53:21.171542 140692488898432 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 1281 into context_free_model/model.ckpt.


I0526 21:12:49.591447 140692488898432 basic_session_run_hooks.py:594] Saving checkpoints for 1281 into context_free_model/model.ckpt.


INFO:tensorflow:Loss for final step: 0.1030399.


I0526 21:13:01.275105 140692488898432 estimator.py:359] Loss for final step: 0.1030399.


Training took time  0:19:50.567569


Now let's use our test data to see how well our model did:

In [24]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0526 21:13:03.059511 140692488898432 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0526 21:13:06.814377 140692488898432 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 1281: auc = 0.7351732, eval_accuracy = 0.7351732, f1_score = 0.7405063, false_negatives = 416.0, false_positives = 486.0, global_step = 1281, loss = 0.86521447, precision = 0.7258883, recall = 0.7557252, true_negatives = 1217.0, true_positives = 1287.0


I0526 21:13:55.470764 140692488898432 estimator.py:1979] Saving dict for global step 1281: auc = 0.7351732, eval_accuracy = 0.7351732, f1_score = 0.7405063, false_negatives = 416.0, false_positives = 486.0, global_step = 1281, loss = 0.86521447, precision = 0.7258883, recall = 0.7557252, true_negatives = 1217.0, true_positives = 1287.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1281: context_free_model/model.ckpt-1281


I0526 21:13:58.716257 140692488898432 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1281: context_free_model/model.ckpt-1281


{'auc': 0.7351732,
 'eval_accuracy': 0.7351732,
 'f1_score': 0.7405063,
 'false_negatives': 416.0,
 'false_positives': 486.0,
 'global_step': 1281,
 'loss': 0.86521447,
 'precision': 0.7258883,
 'recall': 0.7557252,
 'true_negatives': 1217.0,
 'true_positives': 1287.0}

Now let's write code to make predictions on new sentences:

In [0]:
def getPrediction_sent(in_sentences):
  labels = ["Non-sarcastic", "Sarcastic"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = -1) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [
  "Good luck with that!",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "I don't think that's a good idea"
]

In [3]:
predictions = getPrediction_sent(pred_sentences)

NameError: ignored

Voila! We have a sentiment classifier!

In [0]:
predictions

[('Good luck with that!',
  array([-1.9468852 , -0.15398827], dtype=float32),
  'Sarcastic'),
 ('The acting was a bit lacking',
  array([-2.113361  , -0.12877831], dtype=float32),
  'Sarcastic'),
 ('The film was creative and surprising',
  array([-4.3373766 , -0.01315698], dtype=float32),
  'Sarcastic'),
 ("I don't think that's a good idea",
  array([-0.03443514, -3.3858461 ], dtype=float32),
  'Non-sarcastic')]

In [4]:
test_responses2.head()

NameError: ignored

In [0]:
test_responses1[test_responses1.probs2>0]

,Response,Ancestor,Label,probs,labels,probs1,probs2


[-1, 1]
INFO:tensorflow:Writing example 0 of 1703


I0526 21:17:29.020858 140692488898432 run_classifier.py:774] Writing example 0 of 1703


INFO:tensorflow:*** Example ***


I0526 21:17:29.024974 140692488898432 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Running local_init_op.


I0526 21:17:58.234807 140692488898432 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0526 21:17:58.331064 140692488898432 session_manager.py:493] Done running local_init_op.


,Response,Ancestor,Label,probs1,probs2,labels
0,oooh baby you caught me red handed creepin' on...,"wisconsin gop caught red handed, looking to ru...",-1,-0.927477,-0.503437,1
1,welcome to china,i've honestly never come across a dumber human...,-1,-0.022807,-3.792060,-1
2,what's the vegas line on this one?,barney frank and ron paul will introduce legis...,-1,-0.269361,-1.443362,-1
3,"to quote william gibson: ""she stared directly ...",you are all aware that after creating one fina...,-1,-0.015717,-4.160839,-1
4,i wonder why it isn't thought men should have ...,"starting today, 47 million women will have acc...",-1,-4.690615,-0.009223,1
5,it's ok guys it'll make romney richer.,"romney's economic plan would kill 360,000 jobs...",1,-4.581449,-0.010293,1
6,that is a blatant violation of the law on so m...,get tested or get out: louisiana public school...,-1,-0.007694,-4.871174,-1
7,"you sure that wasn't ""boy""?",congressman joe walsh (r-il) calls president o...,-1,-2.805823,-0.062362,1
8,bootstraps of course.,paul ryan tells student he should work three j...,-1,-0.510528,-0.916738,-1
9,sounds like tuesday night at the apollo.,man accidentally shoots himself when concealed...,-1,-0.004147,-5.487556,-1


	Test acc:  0.8073987081620669


In [70]:
labels=tf.convert_to_tensor(test_expect_labels)
preds = tf.convert_to_tensor(test_pred_labels)
a = tf.metrics.auc(labels, preds)
labels, preds

(<tf.Tensor 'Const_8:0' shape=(1703,) dtype=int64>,
 <tf.Tensor 'Const_9:0' shape=(1703,) dtype=int64>)

In [68]:
tensor = tf.range(10)
print(tf.print(tensor))

name: "PrintV2_2"
op: "PrintV2"
input: "StringFormat_2"
attr {
  key: "output_stream"
  value {
    s: "stderr"
  }
}



In [44]:
test_data_with_preds.head(10)

,Response 1,Response 2,Ancestor,Label 1,preds
0,and we're upset since the democrats would *nev...,oooh baby you caught me red handed creepin' on...,"wisconsin gop caught red handed, looking to ru...",1,1
1,"til, voluntary slavery by starving people who'...",welcome to china,i've honestly never come across a dumber human...,1,1
2,that'd be great if only ron paul wasn't a chri...,what's the vegas line on this one?,barney frank and ron paul will introduce legis...,1,1
3,"but, but...lowering taxes creates jobs!","to quote william gibson: ""she stared directly ...",you are all aware that after creating one fina...,1,1
4,whores!,i wonder why it isn't thought men should have ...,"starting today, 47 million women will have acc...",1,-1
5,"well, yeah, but his plan would funnel money to...",it's ok guys it'll make romney richer.,"romney's economic plan would kill 360,000 jobs...",-1,-1
6,"file under ""how the right wing wants to keep g...",that is a blatant violation of the law on so m...,get tested or get out: louisiana public school...,1,-1
7,he has great respect for the office.,"you sure that wasn't ""boy""?",congressman joe walsh (r-il) calls president o...,1,1
8,"i like romney's advice better - ""borrow from y...",bootstraps of course.,paul ryan tells student he should work three j...,1,-1
9,just another responsible gun owner excercising...,sounds like tuesday night at the apollo.,man accidentally shoots himself when concealed...,1,1


In [47]:
test_data.head()

,Response 1,Response 2,Ancestor,Label 1,Label 2
0,and we're upset since the democrats would *nev...,oooh baby you caught me red handed creepin' on...,"wisconsin gop caught red handed, looking to ru...",1,-1
1,"til, voluntary slavery by starving people who'...",welcome to china,i've honestly never come across a dumber human...,1,-1
2,that'd be great if only ron paul wasn't a chri...,what's the vegas line on this one?,barney frank and ron paul will introduce legis...,1,-1
3,"but, but...lowering taxes creates jobs!","to quote william gibson: ""she stared directly ...",you are all aware that after creating one fina...,1,-1
4,whores!,i wonder why it isn't thought men should have ...,"starting today, 47 million women will have acc...",1,-1


In [1]:
test_data_with_preds.to_csv("results_context_free.csv",index=False)

NameError: ignored

In [0]:
list(zip(np.exp(test_responses1['probs1'])*0+1*np.exp(test_responses1['probs2']),np.exp(test_responses2['probs1'])*0+1*np.exp(test_responses2['probs2']),np.exp(test_responses1['probs1'])*0+1*np.exp(test_responses1['probs2'])>np.exp(test_responses2['probs1'])*0+1*np.exp(test_responses2['probs2']),train_data['Label 1']))

[(0.9930434226989746, 0.1610960215330124, True, 1),
 (0.0926213338971138, 0.8715921640396118, False, 1),
 (0.9832752346992493, 0.023240525275468826, True, 1),
 (0.9856531620025635, 0.008691460825502872, True, 1),
 (0.6123374700546265, 0.9902607798576355, False, -1),
 (0.9889251589775085, 0.9848472476005554, True, -1),
 (0.0055563789792358875, 0.010616176761686802, False, 1),
 (0.9875761270523071, 0.5074001550674438, True, 1),
 (0.19164417684078217, 0.5742444396018982, False, 1),
 (0.9915619492530823, 0.012585447169840336, True, -1),
 (0.07511938363313675, 0.9820523858070374, False, -1),
 (0.00808774959295988, 0.0350789949297905, False, 1),
 (0.9886903762817383, 0.9935120940208435, False, 1),
 (0.988135814666748, 0.002830124692991376, True, -1),
 (0.005193168297410011, 0.9879015684127808, False, -1),
 (0.03830774873495102, 0.6014970541000366, False, -1),
 (0.9841346144676208, 0.006175633054226637, True, -1),
 (0.9651614427566528, 0.004968560766428709, True, 1),
 (0.06829016655683517, 0.

In [0]:
l = [[1,2],[3,4]]
import numpy as np
np.array(l)[:,1]

array([2, 4])

In [0]:
test_responses1['probs'][:,1]

ValueError: ignored